In [1]:
%run data_utils.py

In [2]:
import polars as pl
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
from scipy.optimize import brentq

In [3]:
ticker = 'aapl'

In [4]:
#Use only trading days (252) in calculations; no calendar days (365)
df = get_data(ticker)

In [5]:
def print_dict_keys_branching(d, indent=0):
    for key, value in d.items():
        # Print the current key with indentation
        print('    ' * indent + str(key))
        
        # If the value is a dictionary, recurse with increased indentation
        if isinstance(value, dict):
            print_dict_keys_branching(value, indent + 1)

print_dict_keys_branching(df)

stock
    stock
    period
    interval
curr_options
    option_chain
    expiration_date
hist_options


In [6]:
S0 = round(df['stock']['stock']['Close'].iloc[-1], 2)
r = .035
sigma = df['stock']['stock']['Close'].pct_change().std() * np.sqrt(252)
print(f"S0: {S0}, r: {r}, sigma: {sigma}")

S0: 259.35, r: 0.035, sigma: 0.06157765513941486


In [7]:
class VolatilityMeasures:
    def __init__(self, price_series):
        self.price_series = price_series

    def historical_volatility(self, window=60):
        log_returns = np.log(self.price_series / self.price_series.shift(1)).dropna()
        if len(log_returns) < window:
            raise ValueError("Not enough data to compute historical volatility.")
        return log_returns.rolling(window=window).std().iloc[-1] * np.sqrt(252)
    
    def implied_volatility(self, option_chain, S0, X, T, r, q=0.0, option_type='call'):
        def bs_price(sigma):
            d1 = (np.log(S0 / X) + (r - q + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
            d2 = d1 - sigma * np.sqrt(T)
            if option_type == 'call':
                price = S0 * np.exp(-q * T) * norm.cdf(d1) - X * np.exp(-r * T) * norm.cdf(d2)
            elif option_type == 'put':
                price = X * np.exp(-r * T) * norm.cdf(-d2) - S0 * np.exp(-q * T) * norm.cdf(-d1)
            else:
                raise ValueError("option_type must be 'call' or 'put'")
            return price
        
        if 'lastPrice' not in option_chain.columns:
            raise ValueError("Option chain must contain 'lastPrice' column.")

        market_price = option_chain['lastPrice'].iloc[0]
        try:
            implied_vol = brentq(lambda sigma: bs_price(sigma) - market_price, 1e-6, 100, maxiter=100)
        except Exception as e:
            raise ValueError(f"Could not compute implied volatility: {e}")
        return implied_vol

In [8]:
class Greeks:
    def __init__(self, S0, X, T, r, sigma, q=0.0):
        self.S = S0  # Current stock price
        self.X = X    # Strike price
        self.T = T    # Time to expiration in years
        self.r = r    # Risk-free interest rate
        self.sigma = sigma  # Volatility of the underlying stock
        self.q = q    # Dividend yield

    def d1(self):
        return (np.log(self.S / self.X) + (self.r - self.q + .5 * self.sigma ** 2) * self.T) / (self.sigma * np.sqrt(self.T))
    
    def d2(self):
        return self.d1() - self.sigma * np.sqrt(self.T)
    
    def delta(self, option_type='call'):
        d1 = self.d1()
        if option_type == 'call':
            return np.exp(-self.q * self.T) * norm.cdf(d1)
        elif option_type == 'put':
            return np.exp(-self.q * self.T) * (norm.cdf(d1) - 1)
    
    def gamma(self):
        d1 = self.d1()
        return (np.exp(-self.q * self.T) * norm.pdf(d1)) / (self.S * self.sigma * np.sqrt(self.T))
    
    def theta(self, option_type='call'):
        d1 = self.d1()
        d2 = self.d2()
        first_term = - (self.S * norm.pdf(d1) * self.sigma * np.exp(-self.q * self.T)) / (2 * np.sqrt(self.T))

        if option_type == 'call':
            second_term = self.q * self.S * norm.cdf(d1) * np.exp(-self.q * self.T)
            third_term = self.r * self.X * np.exp(-self.r * self.T) * norm.cdf(d2)
            return (first_term - second_term - third_term) / 252
        elif option_type == 'put':
            second_term = self.q * self.S * norm.cdf(-d1) * np.exp(-self.q * self.T)
            third_term = self.r * self.X * np.exp(-self.r * self.T) * norm.cdf(-d2)
            return (first_term + second_term + third_term) / 252
        
    def vega(self):
        d1 = self.d1()
        return self.S * np.exp(-self.q * self.T) * norm.pdf(d1) * np.sqrt(self.T) / 100
    
    def rho(self, option_type='call'):
        d2 = self.d2()
        if option_type == 'call':
            return self.X * self.T * np.exp(-self.r * self.T) * norm.cdf(d2) / 100
        elif option_type == 'put':
            return -self.X * self.T * np.exp(-self.r * self.T) * norm.cdf(-d2) / 100
        
    def all_greeks(self, option_type='call'):
        return {
            'Delta': self.delta(option_type),
            'Gamma': self.gamma(),
            'Theta': self.theta(option_type),
            'Vega': self.vega(),
            'Rho': self.rho(option_type)
        }

In [9]:
class BinomialTreeModel:
    def __init__(self, S0, X, T, r, sigma, N):
        self.S = S0      # Initial stock price
        self.X = X        # Strike price
        self.T = T        # Time to expiration in years
        self.r = r        # Risk-free interest rate
        self.sigma = sigma# Volatility of the underlying stock
        self.N = N        # Number of time steps

        if T <= 0:
            raise ValueError("Time to expiration T must be a positive number.")
        if r < 0:
            raise ValueError("Risk-free interest rate r must be a non-negative number.")
        if sigma < 0:
            raise ValueError("Volatility sigma must be a non-negative number.")
        
        self.dt = T / N   # Time step
        self.u = np.exp(sigma * np.sqrt(self.dt))  # Up factor
        self.d = 1 / self.u                        # Down factor
        self.p = (np.exp(r * self.dt) - self.d) / (self.u - self.d)  # Risk-neutral probability

        if not (0 < self.d < 1 < self.u):
            raise ValueError("Invalid parameters leading to non-positive up/down factors.")
        if not (0 < self.p < 1):
            raise ValueError("Invalid parameters leading to non-positive risk-neutral probability.")
        
    def option_price(self, option_type='call'):

        ST = np.zeros(self.N + 1)
        option_values = np.zeros(self.N + 1)

        for i in range(self.N + 1):
            ST[i] = self.S * (self.u ** (self.N - i)) * (self.d ** i)
        
        if option_type == 'call':
            option_values = np.maximum(0, ST - self.X)
        elif option_type == 'put':
            option_values = np.maximum(0, self.X - ST)
        
        for j in range(self.N - 1, -1, -1):
            for i in range(j + 1):
                option_values[i] = np.exp(-self.r * self.dt) * (self.p * option_values[i+1] + (1 - self.p) * option_values[i])
        
        return option_values[0]

In [10]:
class BlackScholesModel:
    def __init__(self, S0, X, T, r, sigma, q=0.0):
        self.S = S0      # Current stock price
        self.X = X        # Strike price
        self.T = T        # Time to expiration in years
        self.r = r        # Risk-free interest rate
        self.sigma = sigma# Volatility of the underlying stock
        self.q = q        # Dividend yield
    
    def d1(self):
        return (np.log(self.S / self.X) + (self.r - self.q + 0.5 * self.sigma ** 2) * self.T) / (self.sigma * np.sqrt(self.T))
    
    def d2(self):
        return self.d1() - self.sigma * np.sqrt(self.T)
    
    def call_price(self):
        d1 = self.d1()
        d2 = self.d2()
        call_price = (self.S * np.exp(-self.q * self.T) * norm.cdf(d1)) - (self.X * np.exp(-self.r * self.T) * norm.cdf(d2))
        return call_price

    def put_price(self):
        d1 = self.d1()
        d2 = self.d2()
        put_price = (self.X * np.exp(-self.r * self.T) * norm.cdf(-d2)) - (self.S * np.exp(-self.q * self.T) * norm.cdf(-d1))
        return put_price

In [11]:
class Visualizations:
    
    def __init__(self, data):
        self.data = data
    
    def plot_price_history(self, ticker):
        plt.figure(figsize=(10, 6))
        plt.plot(self.data[ticker]['stock'].index, self.data[ticker]['stock']['Close'], label=f'{ticker} Price History')
        plt.title(f'{ticker} Price History')
        plt.xlabel('Date')
        plt.ylabel('Price')
        plt.legend()
        plt.show()
    
    def plot_volatility(self, historical_vol, implied_vol):
        labels = ['Historical Volatility', 'Implied Volatility']
        values = [historical_vol, implied_vol]
        plt.figure(figsize=(10, 6))
        plt.bar(labels, values, color=['blue', 'orange'])
        plt.title('Volatility Comparison')
        plt.ylabel('Volatility')
        plt.show()

    def plot_greeks(self, greeks):
        labels = list(greeks.keys())
        values = list(greeks.values())
        plt.figure(figsize=(10, 6))
        plt.bar(labels, values, color=['green', 'red', 'blue', 'orange', 'purple'])
        plt.title('Option Greeks')
        plt.ylabel('Value')
        plt.show()

In [12]:
class HestonModel:
    def __init__(self, S0, X, T, r, kappa, theta, sigma, v0, rho, num_sims=10000, num_steps=100):
        self.S = S0
        self.X = X
        self.T = T
        self.r = r
        self.kappa = kappa
        self.theta = theta
        self.sigma = sigma
        self.v0 = v0
        self.rho = rho
        self.num_sims = num_sims
        self.num_steps = num_steps
    
    def simulate_price_paths(self):
        dt = self.T / self.num_steps
        S_paths = np.zeros((self.num_sims, self.num_steps + 1))
        v_paths = np.zeros((self.num_sims, self.num_steps + 1))
        S_paths[:, 0] = self.S
        v_paths[:, 0] = self.v0

        for t in range(1, self.num_steps + 1):
            Z1 = np.random.standard_normal(size=self.num_sims)
            Z2 = np.random.standard_normal(size=self.num_sims)
            W1 = np.sqrt(dt) * Z1
            W2 = np.sqrt(dt) * (self.rho * Z1 + np.sqrt(1 - self.rho ** 2) * Z2)

            # Confirm following equations align with Heston model dynamics
            v_paths[:, t] = np.maximum(0, v_paths[:, t-1] + self.kappa * (self.theta - v_paths[:, t-1]) * dt + self.sigma * np.sqrt(v_paths[:, t-1]) * W2)
            S_paths[:, t] = S_paths[:, t-1] * np.exp((self.r - 0.5 * v_paths[:, t-1]) * dt + np.sqrt(v_paths[:, t-1]) * W1)

        return S_paths
    
    def option_price(self, option_type='call'):
        S_paths = self.simulate_price_paths()
        if option_type == 'call':
            payoffs = np.maximum(0, S_paths[:, -1] - self.X)
        elif option_type == 'put':
            payoffs = np.maximum(0, self.X - S_paths[:, -1])
        discounted_payoff = np.exp(-self.r * self.T) * np.mean(payoffs)
        return discounted_payoff

In [13]:
class MertonJumpDiffusionModel:
    def __init__(self, S0, X, T, r, sigma, q=0.0, lambda_=.1, mu=-.2, delta=.1, num_sims=10000, num_steps=100):
        self.S = S0
        self.X = X
        self.T = T
        self.r = r
        self.sigma = sigma
        self.q = q
        self.lambda_ = lambda_
        self.mu = mu
        self.delta = delta
        self.num_sims = num_sims
        self.num_steps = num_steps

    def simulate_price_paths(self):
        dt = self.T / self.num_steps
        S_paths = np.zeros((self.num_sims, self.num_steps + 1))
        S_paths[:, 0] = self.S

        for t in range(1, self.num_steps + 1):
            Z = np.random.standard_normal(size=self.num_sims)
            W = np.sqrt(dt) * Z

            J = np.random.poisson(self.lambda_ * dt, size=self.num_sims)
            Y = np.random.normal(self.mu, self.delta, size=self.num_sims)

            S_paths[:, t] = S_paths[:, t-1] * np.exp((self.r - self.q - 0.5 * self.sigma ** 2) * dt + self.sigma * W + (J * Y))

        return S_paths
    
    def option_price(self, option_type='call'):
        S_paths = self.simulate_price_paths()
        if option_type == 'call':
            payoffs = np.maximum(0, S_paths[:, -1] - self.X)
        elif option_type == 'put':
            payoffs = np.maximum(0, self.X - S_paths[:, -1])
        discounted_payoff = np.exp(-self.r * self.T) * np.mean(payoffs)
        return discounted_payoff

In [14]:
class HullWhiteModel:
    def __init__(self, S0, X, T, r, sigma, kappa, theta, v0, num_sims=10000, num_steps=100):
        self.S = S0
        self.X = X
        self.T = T
        self.r = r
        self.sigma = sigma
        self.kappa = kappa
        self.theta = theta
        self.v = v0
        self.num_sims = num_sims
        self.num_steps = num_steps

    def simulate_price_paths(self):
        dt = self.T / self.num_steps
        S_paths = np.zeros((self.num_sims, self.num_steps + 1))
        v_paths = np.zeros((self.num_sims, self.num_steps + 1))
        S_paths[:, 0] = self.S
        v_paths[:, 0] = self.v

        for t in range(1, self.num_steps + 1):
            Z1 = np.random.standard_normal(size=self.num_sims)
            Z2 = np.random.standard_normal(size=self.num_sims)
            W1 = np.sqrt(dt) * Z1
            W2 = np.sqrt(dt) * Z2

            v_paths[:, t] = np.abs(v_paths[:, t-1] + self.kappa * (self.theta - v_paths[:, t-1]) * dt + self.sigma * np.sqrt(v_paths[:, t-1]) * W2)
            S_paths[:, t] = S_paths[:, t-1] * np.exp((self.r - 0.5 * v_paths[:, t-1]) * dt + np.sqrt(v_paths[:, t-1]) * W1)

        return S_paths
    
    def option_price(self, option_type='call'):
        S_paths = self.simulate_price_paths()
        if option_type == 'call':
            payoffs = np.maximum(0, S_paths[:, -1] - self.X)
        elif option_type == 'put':
            payoffs = np.maximum(0, self.X - S_paths[:, -1])
        discounted_payoff = np.exp(-self.r * self.T) * np.mean(payoffs)
        return discounted_payoff

In [15]:
today = pd.Timestamp('2026-01-09', tz='UTC')

opt_chain_calls = df['curr_options']['option_chain'].calls
opt_chain_calls = pd.DataFrame(opt_chain_calls)
opt_chain_calls = opt_chain_calls[opt_chain_calls['lastTradeDate'] >= today]
opt_chain_calls = opt_chain_calls[opt_chain_calls['volume'] > .001*opt_chain_calls['volume'].max()]
display(opt_chain_calls)

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency
21,AAPL260116C00110000,2026-01-09 20:26:23+00:00,110.0,150.00,148.75,150.15,0.899994,0.603618,349,555,2.570316,True,REGULAR,USD
29,AAPL260116C00150000,2026-01-09 20:04:45+00:00,150.0,110.20,108.80,110.15,1.430000,1.314701,60,57068,1.746095,True,REGULAR,USD
33,AAPL260116C00170000,2026-01-09 20:40:18+00:00,170.0,90.00,88.90,90.10,1.500000,1.694915,258,4975,1.417972,True,REGULAR,USD
38,AAPL260116C00195000,2026-01-09 19:50:23+00:00,195.0,65.15,63.80,65.20,1.300003,2.036027,93,6686,0.998047,True,REGULAR,USD
39,AAPL260116C00200000,2026-01-09 20:53:08+00:00,200.0,59.50,58.85,60.00,1.000000,1.709402,108,21064,0.824221,True,REGULAR,USD
40,AAPL260116C00205000,2026-01-09 18:51:19+00:00,205.0,54.46,53.85,55.20,0.410000,0.758557,69,6443,0.864259,True,REGULAR,USD
43,AAPL260116C00220000,2026-01-09 20:06:20+00:00,220.0,40.00,38.95,40.10,0.500000,1.265823,95,21795,0.634769,True,REGULAR,USD
44,AAPL260116C00225000,2026-01-09 19:55:01+00:00,225.0,34.50,34.35,35.25,0.549999,1.620027,181,24537,0.671878,True,REGULAR,USD
45,AAPL260116C00230000,2026-01-09 20:40:54+00:00,230.0,30.17,28.95,29.90,0.559999,1.891251,354,20841,0.613773,True,REGULAR,USD
46,AAPL260116C00235000,2026-01-09 20:20:34+00:00,235.0,25.28,24.10,25.05,1.480001,6.218494,89,19833,0.558598,True,REGULAR,USD


In [16]:
expiries = pd.Timestamp(df['curr_options']['expiration_date'], tz='UTC')
print((expiries - today).days)
# print()

7


In [17]:
atm_call = opt_chain_calls[opt_chain_calls['inTheMoney'] == True].iloc[-1]
print(atm_call['strike'])

257.5


In [18]:
data = df

price_data = data['stock']['stock']
S0 = round(price_data['Close'].iloc[-1], 2)
strike_prices = opt_chain_calls['strike'].values
r = .035
hist_sigma = price_data['Close'].pct_change().std() * np.sqrt(252)
T = (pd.Timestamp(df['curr_options']['expiration_date'], tz='UTC') - today).days / 252

call_price = None
call_prices = []

In [19]:
model = 'Black-Scholes'
if model == 'Binomial Tree':
    N = 100
    for strike_price in strike_prices:
        binom_model = BinomialTreeModel(S0, strike_price, T, r, hist_sigma, N)
        call_price = round(binom_model.option_price(option_type='call'), 4)
        call_prices.append(call_price)
        # print(f"Binomial Tree Call Price for Strike {strike_price}: {call_price}")
if model == 'Black-Scholes':
    for strike_price in strike_prices:
        bs_model = BlackScholesModel(S0, strike_price, T, r, hist_sigma)
        call_price = round(bs_model.call_price(), 4)
        call_prices.append(call_price)
        # print(f"Black-Scholes Call Price for Strike {strike_price}: {call_price}")
if model == 'Heston':
    # parameters require calibration based on market data
    kappa = 2.0
    theta = hist_sigma ** 2
    sigma = 0.1
    v0 = hist_sigma ** 2
    rho = -0.7
    for strike_price in strike_prices:
        heston_model = HestonModel(S0, strike_price, T, r, kappa, theta, sigma, v0, rho)
        call_price = round(heston_model.option_price(option_type='call'), 4)
        call_prices.append(call_price)
        # print(f"Heston Model Call Price for Strike {strike_price}: {call_price}")
if model == 'Merton Jump-Diffusion':
    lambda_ = 0.1  # Jump intensity
    mu = -0.2      # Mean of jump size
    delta = 0.1    # Std dev of jump size
    for strike_price in strike_prices:
        merton_model = MertonJumpDiffusionModel(S0, strike_price, T, r, hist_sigma, lambda_=lambda_, mu=mu, delta=delta)
        call_price = round(merton_model.option_price(option_type='call'), 4)
        call_prices.append(call_price)
        # print(f"Merton Jump-Diffusion Call Price for Strike {strike_price}: {call_price}")
if model == 'Hull-White':
    kappa = 2.0
    theta = hist_sigma ** 2
    sigma = 0.1
    v0 = hist_sigma ** 2
    for strike_price in strike_prices:
        hull_white_model = HullWhiteModel(S0, strike_price, T, r, hist_sigma, kappa, theta, v0)
        call_price = round(hull_white_model.option_price(option_type='call'), 4)
        call_prices.append(call_price)
        # print(f"Hull-White Model Call Price for Strike {strike_price}: {call_price}")

In [20]:
temp_opt_calls = opt_chain_calls.copy()
temp_opt_calls = temp_opt_calls[['contractSymbol', 'lastTradeDate', 'strike', 'lastPrice', 'bid', 'ask', 'volume', 'inTheMoney']]

In [21]:
temp_opt_calls[model] = call_prices
display(temp_opt_calls)

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,volume,inTheMoney,Black-Scholes
21,AAPL260116C00110000,2026-01-09 20:26:23+00:00,110.0,150.00,148.75,150.15,349,True,149.4569
29,AAPL260116C00150000,2026-01-09 20:04:45+00:00,150.0,110.20,108.80,110.15,60,True,109.4958
33,AAPL260116C00170000,2026-01-09 20:40:18+00:00,170.0,90.00,88.90,90.10,258,True,89.5152
38,AAPL260116C00195000,2026-01-09 19:50:23+00:00,195.0,65.15,63.80,65.20,93,True,64.5395
39,AAPL260116C00200000,2026-01-09 20:53:08+00:00,200.0,59.50,58.85,60.00,108,True,59.5443
40,AAPL260116C00205000,2026-01-09 18:51:19+00:00,205.0,54.46,53.85,55.20,69,True,54.5492
43,AAPL260116C00220000,2026-01-09 20:06:20+00:00,220.0,40.00,38.95,40.10,95,True,39.5638
44,AAPL260116C00225000,2026-01-09 19:55:01+00:00,225.0,34.50,34.35,35.25,181,True,34.5686
45,AAPL260116C00230000,2026-01-09 20:40:54+00:00,230.0,30.17,28.95,29.90,354,True,29.5735
46,AAPL260116C00235000,2026-01-09 20:20:34+00:00,235.0,25.28,24.10,25.05,89,True,24.5784


In [22]:
greeks = Greeks(S0, atm_call['strike'], T, r, hist_sigma)
atm_greeks = greeks.all_greeks(option_type='call')
for key, value in atm_greeks.items():
    print(f"{key}: {value}")

Delta: 0.7873905225604594
Gamma: 0.10906355855994009
Theta: -0.08321685816080865
Vega: 0.12547951021224382
Rho: 0.056051801937156816


In [23]:
greeks_analysis = opt_chain_calls[['contractSymbol', 'strike', 'lastPrice', 'openInterest', 'impliedVolatility']].copy()
for strike in greeks_analysis['strike']:
    greeks_instance = Greeks(S0, strike, T, r, hist_sigma)
    greeks_values = greeks_instance.all_greeks(option_type='call')
    for key, value in greeks_values.items():
        if key not in greeks_analysis.columns:
            greeks_analysis[key] = np.nan
        greeks_analysis.loc[greeks_analysis['strike'] == strike, key] = round(value, 6)

In [24]:
display(greeks_analysis)

,contractSymbol,strike,lastPrice,openInterest,impliedVolatility,Delta,Gamma,Theta,Vega,Rho
21,AAPL260116C00110000,110.0,150.00,555,2.570316,1.000000,0.000000,-0.015263,0.000000,0.030526
29,AAPL260116C00150000,150.0,110.20,57068,1.746095,1.000000,0.000000,-0.020813,0.000000,0.041626
33,AAPL260116C00170000,170.0,90.00,4975,1.417972,1.000000,0.000000,-0.023588,0.000000,0.047176
38,AAPL260116C00195000,195.0,65.15,6686,0.998047,1.000000,0.000000,-0.027057,0.000000,0.054114
39,AAPL260116C00200000,200.0,59.50,21064,0.824221,1.000000,0.000000,-0.027751,0.000000,0.055502
40,AAPL260116C00205000,205.0,54.46,6443,0.864259,1.000000,0.000000,-0.028445,0.000000,0.056889
43,AAPL260116C00220000,220.0,40.00,21795,0.634769,1.000000,0.000000,-0.030526,0.000000,0.061052
44,AAPL260116C00225000,225.0,34.50,24537,0.671878,1.000000,0.000000,-0.031220,0.000000,0.062439
45,AAPL260116C00230000,230.0,30.17,20841,0.613773,1.000000,0.000000,-0.031913,0.000000,0.063827
46,AAPL260116C00235000,235.0,25.28,19833,0.558598,1.000000,0.000000,-0.032607,0.000000,0.065214


In [28]:
hist_options = df['hist_options'].to_pandas()
hist_options = hist_options.sort_values(by=['date', 'expiration'], ascending=False)

In [29]:
print(type(hist_options))

<class 'pandas.core.frame.DataFrame'>


In [30]:
display(hist_options.head())

,date,act_symbol,expiration,strike,call_put,bid,ask,vol,delta,gamma,theta,vega,rho
146814,2026-01-09,AAPL,2026-02-27,180.0,Call,78.20,81.84,0.4458,0.9899,0.0006,-0.0272,0.0245,0.2368
146815,2026-01-09,AAPL,2026-02-27,180.0,Put,0.00,0.79,0.5837,-0.0335,0.0013,-0.0412,0.0707,-0.0127
146816,2026-01-09,AAPL,2026-02-27,195.0,Call,63.30,67.25,0.4040,0.9782,0.0013,-0.0374,0.0487,0.2525
146817,2026-01-09,AAPL,2026-02-27,195.0,Put,0.01,2.19,0.6013,-0.0774,0.0025,-0.0827,0.1448,-0.0279
146818,2026-01-09,AAPL,2026-02-27,205.0,Call,53.55,57.15,0.3477,0.9733,0.0018,-0.0387,0.0579,0.2642


In [38]:
curr_expiry = expiries.date().strftime('%Y-%m-%d')
print(curr_expiry)

2026-01-16


In [45]:
pivot_table = pd.pivot_table(hist_options,
                             values='strike',
                             index=['date'],
                             columns=['expiration'],
                             aggfunc='count',
                             fill_value=0)
display(pivot_table)

expiration,2019-02-22,2019-03-01,2019-03-08,2019-03-15,2019-03-22,2019-03-29,2019-04-05,2019-04-12,2019-04-18,2019-04-26,...,2025-12-26,2026-01-02,2026-01-09,2026-01-16,2026-01-23,2026-01-30,2026-02-06,2026-02-13,2026-02-20,2026-02-27
date,,,,,,,,,,,,,,,,,,,,,
2019-02-09,22,0,22,0,0,22,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2019-02-16,0,22,0,18,0,0,0,0,18,0,...,0,0,0,0,0,0,0,0,0,0
2019-02-23,0,0,22,0,22,0,0,0,20,0,...,0,0,0,0,0,0,0,0,0,0
2019-03-02,0,0,0,22,0,22,0,0,18,0,...,0,0,0,0,0,0,0,0,0,0
2019-03-09,0,0,0,0,22,0,22,0,0,22,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2026-01-05,0,0,0,0,0,0,0,0,0,0,...,0,0,0,54,0,48,0,48,48,0
2026-01-06,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,54,0,48,0,48,0
2026-01-07,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,54,0,46,0,46,0


In [41]:
filtered_hist_options = hist_options[hist_options['expiration'] == curr_expiry]
filtered_hist_options = filtered_hist_options.sort_values(by='date', ascending=False)
display(filtered_hist_options)

,date,act_symbol,expiration,strike,call_put,bid,ask,vol,delta,gamma,theta,vega,rho
145982,2026-01-05,AAPL,2026-01-16,185.0,Call,81.84,83.20,0.8061,0.9958,0.0002,-0.0177,0.0047,0.0554
146024,2026-01-05,AAPL,2026-01-16,300.0,Call,0.03,0.04,0.2722,0.0084,0.0018,-0.0132,0.0106,0.0006
146012,2026-01-05,AAPL,2026-01-16,272.5,Call,1.70,1.76,0.1951,0.3044,0.0386,-0.1466,0.1625,0.0240
146013,2026-01-05,AAPL,2026-01-16,272.5,Put,6.55,6.75,0.2002,-0.6920,0.0382,-0.1434,0.1619,-0.0456
146014,2026-01-05,AAPL,2026-01-16,275.0,Call,1.07,1.11,0.1938,0.2157,0.0325,-0.1215,0.1359,0.0170
...,...,...,...,...,...,...,...,...,...,...,...,...,...
138771,2025-11-11,AAPL,2026-01-16,260.0,Put,4.35,4.50,0.2426,-0.2494,0.0113,-0.0607,0.3738,-0.1135
138772,2025-11-11,AAPL,2026-01-16,265.0,Call,18.40,18.60,0.2426,0.6937,0.0123,-0.0968,0.4117,0.3121
138773,2025-11-11,AAPL,2026-01-16,265.0,Put,5.70,5.85,0.2373,-0.3095,0.0129,-0.0650,0.4175,-0.1374
138774,2025-11-11,AAPL,2026-01-16,270.0,Call,15.10,15.30,0.2378,0.6291,0.0135,-0.0992,0.4432,0.2859
